# Linear Prediction Coefficient

### Introduction

Linear Predictive Coding(LPC) provides an accurate and economical representation of relevant speech parameters that can reduce transmission rates in speech coding, increase accuracy and reduce calculation in speech recognition, and generate efficient speech synthesis. 

The basic idea behind LPC is that a speech sample can be approximated as a linear combination of past speech samples. By minimizing the sum of squared differences over a finite interval between actual speech samples and the linearly predicted ones, a unique set of predictor coefficients can be determined.

### Setup

We first need to import the required Python libraries for the entire anaylsis  
Prerequisites required in this implementation are:
```
Python 3.x 
numpy   
scipy
librosa
```

You can invoke any required library into your workspace as:  
`import package_name`  

In [1]:
import numpy as np
import scipy.signal as sig
import scipy.io.wavfile as wav
import scipy.linalg as linalg
import librosa
from scipy.linalg import solve_toeplitz

Reading the file name, and setting the order of the LPC model as 13

In [2]:
filename = "audio_files/spectrogram_audio.wav"
sample_rate, signal = wav.read(filename)
order = 13

### Preprocessing
Applying pre-emphasis and windowing

In [3]:
pre_emphasis = 0.97
emphasized_signal = np.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])
frame_size = 0.025 
frame_stride = 0.01
frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate  # Convert from seconds to samples
signal_length = len(emphasized_signal)
frame_length = int(round(frame_length))
frame_step = int(round(frame_step))
num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame
pad_signal_length = num_frames * frame_step + frame_length
z = np.zeros((pad_signal_length - signal_length))
pad_signal = np.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal
indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
frames = pad_signal[indices.astype(np.int32, copy=False)]
frames *= np.blackman(frame_length)

### Computing LPC for 1 frame
Using Autocorrelation method to generate a toeplitz matrix and solving it using Levsion-Durbin Recursion

In [4]:
autocorrelation = sig.fftconvolve(frames[0], frames[0][::-1])
autocorr_coefficients = autocorrelation[int(len(autocorrelation)/2):][:(order + 1)]
lpc_coefficients = -1*solve_toeplitz(autocorr_coefficients[:order],autocorr_coefficients[1:order+1])
print(lpc_coefficients)

[ 0.21964622 -0.05056925  0.03951827 -0.04997397 -0.19916233 -0.05241685
  0.00071193 -0.16961096 -0.01992969  0.0740895  -0.04449198 -0.07419442
  0.08100477]


Using librosa library to compute the same LPC coefficients

In [5]:
lpc_librosa = librosa.lpc(frames[0],order=13)[1:]
print(lpc_librosa)

[ 0.21964607 -0.05056932  0.03951846 -0.0499739  -0.19916262 -0.05241687
  0.00071183 -0.16961147 -0.01992988  0.07408958 -0.04449215 -0.07419442
  0.08100641]


Making this whole flow into a function

In [6]:
def compute_lpc_coeff(filename, order=13, write_file = False):
    sample_rate, signal = wav.read(filename)
    pre_emphasis = 0.97
    emphasized_signal = np.append(signal[0], signal[1:] - pre_emphasis * signal[:-1])
    frame_size = 0.025 
    frame_stride = 0.01
    frame_length, frame_step = frame_size * sample_rate, frame_stride * sample_rate  # Convert from seconds to samples
    signal_length = len(emphasized_signal)
    frame_length = int(round(frame_length))
    frame_step = int(round(frame_step))
    num_frames = int(np.ceil(float(np.abs(signal_length - frame_length)) / frame_step))  # Make sure that we have at least 1 frame
    pad_signal_length = num_frames * frame_step + frame_length
    z = np.zeros((pad_signal_length - signal_length))
    pad_signal = np.append(emphasized_signal, z) # Pad Signal to make sure that all frames have equal number of samples without truncating any samples from the original signal
    indices = np.tile(np.arange(0, frame_length), (num_frames, 1)) + np.tile(np.arange(0, num_frames * frame_step, frame_step), (frame_length, 1)).T
    
    frames = pad_signal[indices.astype(np.int32, copy=False)]
    frames *= np.blackman(frame_length)
    lpc_coefficients = np.zeros((num_frames,order))
    lpc_librosa = np.zeros((num_frames,order))
    
    for i in range(0,len(frames),1):
        autocorrelation = sig.fftconvolve(frames[i], frames[i][::-1])
        autocorr_coefficients = autocorrelation[int(len(autocorrelation)/2):]
        lpc_coefficients[i] = solve_toeplitz(autocorr_coefficients[:order],autocorr_coefficients[1:order+1])
        lpc_coefficients[i]*= -1
        
        lpc_librosa[i] = librosa.lpc(frames[i],order)[1:]
        
    error = np.sum(np.abs(lpc_librosa - lpc_coefficients))
    print("Error per frame = ", error/num_frames)
    
    if write_file:
        np.savetxt("LPC_coeff.txt",lpc_coefficients)
        
    return lpc_coefficients

In [8]:
lpc_coeff = compute_lpc_coeff("audio_files/spectrogram_audio.wav", 13, True)

Error per frame =  3.1781595209888165e-05
